In [1]:
# Load TensorBoard extension
%load_ext tensorboard

# Start TensorBoard
from datetime import datetime
import os

# Set up a log directory for TensorBoard
log_dir = os.path.join("logs", datetime.now().strftime("%Y%m%d-%H%M%S"))
os.makedirs(log_dir, exist_ok=True)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#INPORT DATA AND PREPROCESS

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import zscore
import os
import cv2
from torch.utils.data import DataLoader, random_split
import torch.optim as optim
import torchvision.utils
from torchvision import datasets, transforms
import numpy as np
def preProcessing():
    LABELS_PATH = '/content/drive/My Drive/OCTDL_labels.csv'
    INPUT_PATH = '/content/drive/My Drive/OCTDL_AUGMENTED'
    OUTPUT_PATH = '/content/drive/My Drive/OCTDL_DCGAN_AUGMENTED'

    df = pd.read_csv(LABELS_PATH)

    dimensions = pd.DataFrame(df, columns = ['file_name', 'image_width', 'image_hight'])

    dimensions['z_width'] = zscore(dimensions['image_width'])
    dimensions['z_height'] = zscore(dimensions['image_hight'])

    z_threshold = 2.5

    filtered_dimensions = dimensions[
        (abs(dimensions['z_width']) <= z_threshold) &
        (abs(dimensions['z_height']) <= z_threshold)
    ].copy()


    filtered_dimensions['aspect_ratio'] = filtered_dimensions['image_width'] / filtered_dimensions['image_hight']
    avg_aspect_ratio = filtered_dimensions['aspect_ratio'].mean()

    # We are using A DCGAN
    # This *REQUIRES* 64x64 by default
    # there /is/ a way around this (see https://github.com/pytorch/examples/issues/70)
    # but this is kind of elaborate and for now will not be implemented.
    # TODO: come back and implement it.
    # Another alterative would be to go back to SRGAN and upscale the images?
    height = 64
    width = 64
    target_size = (width, height)

    def resize_and_save(classname):
        input_dir = INPUT_PATH + '/augmented_data/' + classname
        output_dir = OUTPUT_PATH + '/' + classname

        os.makedirs(output_dir, exist_ok = True)

        for _, row in filtered_dimensions.iterrows():
                file_name = row['file_name'] + '.jpg'
                if classname.lower() not in file_name:
                    continue
                image_path = os.path.join(input_dir, file_name)

                # Read the image
                img = cv2.imread(image_path)
                if img is None:
                    print(f"Error loading image {file_name}")
                    continue

                resized_img = cv2.resize(img, target_size, interpolation = cv2.INTER_AREA)

                output_path = os.path.join(output_dir, file_name)
                cv2.imwrite(output_path, resized_img)

    classnames = ['AMD', 'DME', 'ERM', 'NO', 'RAO', 'RVO', 'VID']

    #for classname in classnames:
    #    resize_and_save(classname)


    # hyperparameters - to be adjusted
    # currently 70/30 split
    TEST_TRAIN_SPLIT = 0.3
    # we'll try 32 for now
    BATCH_SIZE = 32

    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor()
    ])

    num_channels = 1 # used later when declaring generator/descriminator
    features_discrim = 16
    # 64 by 64 images
    dataset = datasets.ImageFolder(OUTPUT_PATH, transform=transform)

    #full preprocessed images
    #dataset = datasets.ImageFolder(INPUT_PATH, transform=transform)

    n_test = int(np.floor(TEST_TRAIN_SPLIT * len(dataset)))
    n_train = len(dataset) - n_test

    train_ds, test_ds = random_split(dataset, [n_train, n_test])

    train_dl = DataLoader(train_ds, batch_size = BATCH_SIZE, shuffle = True, num_workers = 4)
    test_dl = DataLoader(test_ds, batch_size = BATCH_SIZE, shuffle = False, num_workers = 4)


    # some useful info about the dataset
    print(f"Classes: {dataset.classes}")
    print(f"Number of training samples: {len(train_ds)}")
    print(f"Number of testing samples: {len(test_ds)}")
    #for i, (x, label) in enumerate(train_dl):
        #print(label)
        #break
        #print(f"Image shape: {x.shape}")
    return train_dl, test_dl

#Declaring Generator

In [4]:
import torch
import torch.nn as nn
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        num_channels = 1
        features_gen = 16
        self.main = nn.Sequential(

            # input is Z, going into a convolution
            nn.ConvTranspose2d( 100, features_gen * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(features_gen * 8),
            nn.ReLU(True),
            # state size. ``(features_gen*8) x 4 x 4``
            nn.ConvTranspose2d(features_gen * 8, features_gen * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_gen * 4),
            nn.ReLU(True),
            # state size. ``(features_gen*4) x 8 x 8``
            nn.ConvTranspose2d( features_gen * 4, features_gen * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_gen * 2),
            nn.ReLU(True),
            # state size. ``(features_gen*2) x 16 x 16``
            nn.ConvTranspose2d( features_gen * 2, features_gen, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_gen),
            nn.ReLU(True),
            # state size. ``(features_gen) x 32 x 32``
            nn.ConvTranspose2d( features_gen, num_channels, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. ``(num_channels) x 64 x 64``
        )

    def forward(self, input):
        return self.main(input)

device = "cuda" if torch.cuda.is_available() else "cpu"
# Create the generator
netG = Generator(1).to(device)

# Handle multi-GPU if desired
#if (device.type == 'cuda') and (ngpu > 1):
#    netG = nn.DataParallel(netG, list(range(ngpu)))


def initialize_weights(model):
    # Initializes weights according to the DCGAN paper
    for m in model.modules():
        if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
            nn.init.normal_(m.weight.data, 0.0, 0.02)


# Apply the ``weights_init`` function to randomly initialize all weights
#  to ``mean=0``, ``stdev=0.02``.
netG.apply(initialize_weights)

# Print the model
print(netG)

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 128, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(32, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(16, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)


#Declaring Discriminator

In [5]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        num_channels = 1
        features_discrim = 16
        self.main = nn.Sequential(
            # input is ``(num_channels) x 64 x 64``
            nn.Conv2d(num_channels, features_discrim, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(features_discrim) x 32 x 32``
            nn.Conv2d(features_discrim, features_discrim * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_discrim * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(features_discrim*2) x 16 x 16``
            nn.Conv2d(features_discrim * 2, features_discrim * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_discrim * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(features_discrim*4) x 8 x 8``
            nn.Conv2d(features_discrim * 4, features_discrim * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features_discrim * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. ``(features_discrim*8) x 4 x 4``
            nn.Conv2d(features_discrim * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

device = "cuda" if torch.cuda.is_available() else "cpu"
# Create the Discriminator
netD = Discriminator(1).to(device)

# Handle multi-GPU if desired
#if (device.type == 'cuda') and (ngpu > 1):
#    netD = nn.DataParallel(netD, list(range(ngpu)))

# Apply the ``initialize_weights`` function to randomly initialize all weights
# like this: ``to mean=0, stdev=0.2``.
netD.apply(initialize_weights)

# Print the model
print(netD)

Discriminator(
  (main): Sequential(
    (0): Conv2d(1, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(16, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(128, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)


#initialise loss function and optimisers

In [6]:
import torch.optim as optim
# Initialize the ``BCELoss`` function
criterion = nn.BCELoss()

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(64, 100, 1, 1, device=device)

# Establish convention for real and fake labels during training
real_label = 1.
fake_label = 0.

LEARNING_RATE = 0.0002
# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))

#Begin Training

In [7]:
# Training Loop

# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0
num_epochs = 200

train_d1, test_d1, = preProcessing()
dataloader = train_d1

print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(train_d1, 0):

        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        netD.zero_grad()
        # Format batch
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
        # Forward pass real batch through D
        output = netD(real_cpu).view(-1)
        # Calculate loss on all-real batch
        errD_real = criterion(output, label)
        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = output.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = torch.randn(b_size, 100, 1, 1, device=device)
        # Generate fake image batch with G
        fake = netG(noise)
        label.fill_(fake_label)
        # Classify all fake batch with D
        output = netD(fake.detach()).view(-1)
        # Calculate D's loss on the all-fake batch
        errD_fake = criterion(output, label)
        # Calculate the gradients for this batch, accumulated (summed) with previous gradients
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Compute error of D as sum over the fake and the real batches
        errD = errD_real + errD_fake
        # Update D
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = netD(fake).view(-1)
        # Calculate G's loss based on this output
        errG = criterion(output, label)
        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()

        # Output training stats
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            img_list.append(torchvision.utils.make_grid(fake, padding=2, normalize=True))

        iters += 1

Classes: ['AMD', 'DME', 'ERM', 'NO', 'RAO', 'RVO', 'VID']
Number of training samples: 1440
Number of testing samples: 616
Starting Training Loop...


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


[0/200][0/45]	Loss_D: 1.3852	Loss_G: 0.6934	D(x): 0.5006	D(G(z)): 0.5000 / 0.4999
[1/200][0/45]	Loss_D: 0.7146	Loss_G: 1.1868	D(x): 0.7098	D(G(z)): 0.3103 / 0.3052
[2/200][0/45]	Loss_D: 0.2870	Loss_G: 1.9877	D(x): 0.8757	D(G(z)): 0.1429 / 0.1370
[3/200][0/45]	Loss_D: 0.1390	Loss_G: 2.6587	D(x): 0.9392	D(G(z)): 0.0734 / 0.0703
[4/200][0/45]	Loss_D: 0.7916	Loss_G: 1.5903	D(x): 0.6921	D(G(z)): 0.3185 / 0.2244
[5/200][0/45]	Loss_D: 0.7057	Loss_G: 2.1107	D(x): 0.7128	D(G(z)): 0.2908 / 0.1290
[6/200][0/45]	Loss_D: 0.5245	Loss_G: 1.4012	D(x): 0.7432	D(G(z)): 0.1782 / 0.2805
[7/200][0/45]	Loss_D: 0.5081	Loss_G: 2.4682	D(x): 0.7543	D(G(z)): 0.1781 / 0.0896
[8/200][0/45]	Loss_D: 0.4484	Loss_G: 2.2584	D(x): 0.8417	D(G(z)): 0.2240 / 0.1132
[9/200][0/45]	Loss_D: 0.7545	Loss_G: 2.0238	D(x): 0.6502	D(G(z)): 0.2186 / 0.1532
[10/200][0/45]	Loss_D: 0.5215	Loss_G: 2.5366	D(x): 0.8122	D(G(z)): 0.2358 / 0.0891
[11/200][0/45]	Loss_D: 0.3932	Loss_G: 3.0509	D(x): 0.9296	D(G(z)): 0.2626 / 0.0513
[12/200][0/45]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7c9b37dbbe20>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1582, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.11/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/connection.py", line 948, in wait
    ready = selector.select(timeout)
            ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/selectors.py", line 415, in select
    fd_event_list = self._selector.poll(timeout)
                    ^^^^^^^^^^^^^^

KeyboardInterrupt: 

#Plot Results

actually save the models you absolute complete and total moron

In [ ]:
torch.save(netG.state_dict, '/content/drive/My Drive/OCTDL_DCGAN_AUGMENTED/netG.pth')
torch.save(netD.state_dict, '/content/drive/My Drive/OCTDL_DCGAN_AUGMENTED/netD.pth')
torch.save(netG, '/content/drive/My Drive/OCTDL_DCGAN_AUGMENTED/netGfull.pth')
torch.save(netD, '/content/drive/My Drive/OCTDL_DCGAN_AUGMENTED/netDfull.pth')
netG.eval()
netD.eval()

example comparison

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

# Grab a batch of real images from the dataloader
real_batch = next(iter(dataloader))

# Plot the real images
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(torchvision.utils.make_grid(real_batch[0].to(device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))

# Plot the fake images from the last epoch
plt.subplot(1,2,2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(img_list[-1],(1,2,0)))
plt.show()

Alright now we got asked for 2056 example images so we making them

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [16]:
NewG = torch.load('/content/drive/My Drive/OCTDL_DCGAN_AUGMENTED/netGfull.pth',weights_only=False)
#NewG = Generator(1).to(device)
#NewG.load_state_dict(torch.load('/content/drive/My Drive/OCTDL_DCGAN_AUGMENTED/netG.pth', weights_only=False))
NewG.eval()

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 128, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(32, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(16, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

In [21]:
#noise = torch.randn(1, 100, 1, 1, device=device)
#fake = NewG(noise)
fake = np.load('/content/drive/My Drive/OCTDL_DCGAN_AUGMENTED/AMD/amd_1047099_1.jpg')
plt.figure(figsize=(64,64))
plt.imshow(np.transpose(torchvision.utils.make_grid(fake.detach().cpu(), padding=2, normalize=True),(1,2,0)))
plt.show()

UnpicklingError: Failed to interpret file '/content/drive/My Drive/OCTDL_DCGAN_AUGMENTED/AMD/amd_1047099_1.jpg' as a pickle

In [ ]:
for counter in range(2056):
  noise = torch.randn(1, 100, 1, 1, device=device)
  fake = netG(noise)
  torchvision.utils.save_image(fake.detach(), '%s/%d.png' % ("/content/drive/My Drive/OCTDL_DCGAN_AUGMENTED/ImageGen2056", counter), normalize=True)
#noise = torch.randn(256, 100, 1, 1, device=device)
#fake = netG(noise)
#torchvision.utils.save_image(fake.detach(), '%s/2.png' % ("/content/drive/My Drive/OCTDL_DCGAN_AUGMENTED", epoch), normalize=True)
#plt.figure(figsize=(64,64))
#plt.imshow(np.transpose(torchvision.utils.make_grid(fake.detach().cpu(), padding=2, normalize=True),(1,2,0)))
#plt.show()